# Normalización de nombres de pila con lógica difusa

- Sobre [lógica difusa](https://es.wikipedia.org/wiki/Lógica_difusa)
- Sobre [distancia Levenshtein](https://es.wikipedia.org/wiki/Distancia_de_Levenshtein)
- Sobre la [Ley de Zipf](https://es.wikipedia.org/wiki/Ley_de_Zipf)
- Listado de [nombres de pila](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177009&menu=resultados&idp=1254734710990#!tabs-1254736195454)

In [15]:
import re
import time
import pandas as pd
from fuzzywuzzy import process

## Obtención de datos

In [16]:
WOMEN= pd.read_csv('./data/women.txt', delimiter='\t')
WOMEN.head(5)

,Orden,Nombre,Frecuencia,Edad Media (*)
0,1,MARIA CARMEN,656276,"57,0"
1,2,MARIA,606048,"48,6"
2,3,CARMEN,391563,"60,4"
3,4,JOSEFA,276682,"68,0"
4,5,ANA MARIA,273319,"51,2"


In [17]:
MEN= pd.read_csv('./data/men.txt', delimiter='\t')
MEN.head(10)

,Orden,Nombre,Frecuencia,Edad Media (*)
0,1,ANTONIO,678425,"55,9"
1,2,JOSE,594144,"61,1"
2,3,MANUEL,590965,"54,9"
3,4,FRANCISCO,498934,"57,4"
4,5,DAVID,365196,"30,5"
5,6,JUAN,346867,"55,7"
6,7,JOSE ANTONIO,310134,"49,6"
7,8,JAVIER,306504,"33,0"
8,9,DANIEL,296281,"27,0"
9,10,JOSE LUIS,293564,"53,3"


## Normalización

In [18]:
name = 'Mª JOSE'
suggestions = process.extract(name, WOMEN.Nombre.array, limit=3)
suggestions

[('MIRIAM JOSEFINA', 90), ('MIRIAM JOSE', 90), ('MARIA JOSE', 86)]

## Heurística

In [19]:
suggestion_names = [suggestion[0] for suggestion in suggestions]
suggestion_names

['MIRIAM JOSEFINA', 'MIRIAM JOSE', 'MARIA JOSE']

In [20]:
suggestion_rows = WOMEN[WOMEN.Nombre.isin(suggestion_names)]
suggestion_rows

,Orden,Nombre,Frecuencia,Edad Media (*)
17,18,MARIA JOSE,203283,"46,1"
13205,13206,MIRIAM JOSEFINA,42,"54,0"
16750,16751,MIRIAM JOSE,32,"41,7"


In [22]:
suggestion_names = [suggestion[0] for suggestion in suggestions]
suggestion_rows = WOMEN[WOMEN.Nombre.isin(suggestion_names)]
suggestion_rows.Nombre.array[0]

<PandasArray>
['MARIA JOSE', 'MIRIAM JOSEFINA', 'MIRIAM JOSE']
Length: 3, dtype: object

In [24]:
def normalize(name, candidates_df, limit=3):
    candidates = candidates_df.Nombre.array
    suggestions = process.extract(name, candidates, limit=limit)
    suggestion_names = [suggestion[0] for suggestion in suggestions]
    suggestion_rows = candidates_df[candidates_df.Nombre.isin(suggestion_names)]
    best_suggestion = suggestion_rows.Nombre.array[0]
    return best_suggestion

In [27]:
name = 'Mª JOSE'
normalize(name, WOMEN, limit=3)

'MARIA JOSE'

## Optimización

In [25]:
WOMEN.shape

(25658, 4)

In [26]:
name = 'JOSE ANTONO'
start = time.time()
suggestions = process.extract(name, MEN.Nombre.array, limit=3)
end = time.time()
end - start

0.9954941272735596

In [27]:
WOMEN = WOMEN[WOMEN.Frecuencia > 99]
MEN = MEN[MEN.Frecuencia > 99]

In [28]:
name = 'JOSE ANTONO'
start = time.time()
suggestions = process.extract(name, MEN.Nombre.array, limit=3)
end = time.time()
end - start

0.2753720283508301

In [29]:
MEN.shape

(6795, 4)

In [30]:
MEN.tail(10)

,Orden,Nombre,Frecuencia,Edad Media (*)
6785,6786,SADDIK,100,"43,7"
6786,6787,SANTIAGO AGUSTIN,100,"35,1"
6787,6788,SANTIAGO JOAQUIN,100,"43,1"
6788,6789,SIDI AHMED,100,"42,9"
6789,6790,SOHAYB,100,"7,6"
6790,6791,TALAL,100,"30,7"
6791,6792,VICENT JOSEP,100,"45,3"
6792,6793,WILLIAM ALBERTO,100,"39,8"
6793,6794,YADEL,100,"4,6"
6794,6795,ZHIWEI,100,"29,0"


## Adaptación del input

In [31]:
name = 'María'
suggestions = process.extract(name, WOMEN.Nombre.array, limit=3)
print(suggestions)

[('MARA', 100), ('TAMARA', 90), ('SAMARA', 90)]


In [32]:
WOMEN[WOMEN.Nombre.str.contains('MARIA', na=False)].head(10)

,Orden,Nombre,Frecuencia,Edad Media (*)
0,1,MARIA CARMEN,656276,"57,0"
1,2,MARIA,606048,"48,6"
4,5,ANA MARIA,273319,"51,2"
6,7,MARIA PILAR,263141,"57,0"
7,8,MARIA DOLORES,259216,"56,6"
9,10,MARIA TERESA,251492,"57,1"
12,13,MARIA ANGELES,226047,"55,4"
16,17,MARIA ISABEL,204354,"52,8"
17,18,MARIA JOSE,203283,"46,1"
23,24,MARIA LUISA,160565,"60,9"


In [36]:
def adapt(name):
    name = name.lower()
    name = name.translate(str.maketrans({'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'ñ': 'n'}))
    name = re.sub('\s+del?(\s+l[oa]s?)?\s+', ' ', name)
    return name

In [37]:
adapt('María')

'maria'

In [2]:
def normalize(name, candidates_df, limit=3):

    def adapt(name):
        name = name.lower()
        name = name.translate(str.maketrans({'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'ñ': 'n'}))
        name = re.sub('\s+del?(\s+l[oa]s?)?\s+', ' ', name)
        return name

    suggestions = process.extract(adapt(name), candidates_df.Nombre.array, limit=3)
    if suggestions[0][1] == 100:
        return suggestions[0][0]
    suggestion_names = [suggestion[0] for suggestion in suggestions]
    suggestion_rows = candidates_df[candidates_df.Nombre.isin(suggestion_names)]
    best_suggestion = suggestion_rows.Nombre.array[0]

    return best_suggestion
